Since the uprising of Artificial Intelligence, text classification has become one of the most staggering tasks to accomplish. In layman terms, We can say Artificial Intelligence is the field which tries to achieve human-like intelligent models to ease the jobs for all of us. We have an astounding proficiency in text classification but even many sophisticated NLP models are failed to achieve proficiency even close to it. So the question arises is that what we humans do differently? How do we classify text?

First of all, we understand words not each and every word but many of them and we can guess even unknown words just by the structure of a sentence. Then we understand the message that those series of words (sentence) conveys. Then from those series of sentences, we understand the meaning of a paragraph or an article. The similar approach is used in Hierarchical Attention model.

**To Learn Hierarchical Attention Network completely go [here](https://medium.com/@heetsankesara3/hierarchical-attention-networks-d220318cf87e)**

** Click [here](https://github.com/Hsankesara/DeepResearch/tree/master/Hierarchical_Attention_Network) to go to the code**


In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer,  text_to_word_sequence
from tensorflow.keras.layers import Layer, InputSpec

from tensorflow.keras import initializers as initializers, regularizers, constraints
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed, Dropout
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
import nltk
import re
import matplotlib.pyplot as plt
import sys
from sklearn.metrics import roc_auc_score
from nltk import tokenize
import seaborn as sns

In [ ]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatibl|e with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

In [ ]:
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras import initializers, regularizers, constraints

def dot_product(x, kernel):
    """
    https://github.com/richliao/textClassifier/issues/13#issuecomment-377323318
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    
    Note: The layer has been tested with Keras 2.0.6
    
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """
    
    def __init__(self, return_coefficients=False,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform')
        
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
        
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)
    
    def build(self, input_shape):
        assert len(input_shape) == 3
        
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        
        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)
        
        super(AttentionWithContext, self).build(input_shape)
    
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None
    
    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
        
        if self.bias:
            uit += self.b
        
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
        
        a = K.exp(ait)
        
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        
        a = K.expand_dims(a)
        weighted_input = x * a
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), a]
        else:
            return K.sum(weighted_input, axis=1)
    
    
    
    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]
    def get_config(self):
        base_config = super(AttentionWithContext, self).get_config()
        #base_config['output_dim'] = self.output_dim
        return base_config

### Attention Layer

### Config

In [ ]:
max_features=200000
max_senten_len=40
max_senten_num=6
embed_size=100
VALIDATION_SPLIT = 0.2

### Data

In [ ]:
from sklearn.utils import shuffle

In [ ]:
df = shuffle(pd.read_json('E:/saves/News_Category_Dataset_v2.json', lines=True)).reset_index()

In [ ]:
len(df.category.unique())

In [ ]:
df.head()

In [ ]:
df.category.unique()

In [ ]:
df.category = df.category.map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)

In [ ]:
df['text'] = df['headline'] +'. ' +df['short_description']

In [ ]:
df.head()

In [ ]:
df['text'][0]

In [ ]:
df = df[['text', 'category']]

In [ ]:
#df.info()

In [ ]:
categories = df['category']
text = df['text']

In [ ]:
text

In [ ]:
cates = df.groupby('category')
print("total categories:", cates.ngroups)
print(cates.size())

In [ ]:
import re
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [ ]:
paras = []
labels = []
texts = []

In [ ]:
df.text.shape[0]

In [ ]:
#df2 = pd.DataFrame(columns=['text'])

In [ ]:
df2

In [ ]:
df2 = pd.DataFrame([["welcome to florida"]], index = [["text"]])

In [ ]:
df2

In [ ]:
sent_lens = []
sent_nums = []
for idx in range(df.text.shape[0]):
    text = clean_str(df.text[idx])
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    sent_nums.append(len(sentences))
    for sent in sentences:
        sent_lens.append(len(text_to_word_sequence(sent)))
    paras.append(sentences)

In [ ]:
sent_lens = []
sent_nums = []
for idx in range(1):
    text = clean_str(df.text[idx])
    print(text)
    texts.append(text)
    #print(texts)
    sentences = tokenize.sent_tokenize(text)
    print(sentences)
    sent_nums.append(len(sentences))
    for sent in sentences:
        sent_lens.append(len(text_to_word_sequence(sent)))
        print(sent)
    paras.append(sentences)
    

In [ ]:
sentences

In [ ]:
sent_lens

In [ ]:
sns.distplot(sent_lens, bins=200)
plt.show()

In [ ]:
sns.distplot(sent_nums)
plt.show()

In [ ]:
tokenizer = Tokenizer(num_words=max_features, oov_token=True)
tokenizer.fit_on_texts(texts)

In [ ]:
texts

In [ ]:
data = np.zeros((len(texts), max_senten_num, max_senten_len), dtype='int32')
for i, sentences in enumerate(paras):
    for j, sent in enumerate(sentences):
        if j< max_senten_num:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                try:
                    if k<max_senten_len and tokenizer.word_index[word]<max_features:
                        data[i,j,k] = tokenizer.word_index[word]
                        k=k+1
                except:
                    print(word)
                    pass

In [ ]:
p=[55,6,8,12,3,54]
p[:-3]

In [ ]:
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

In [ ]:
labels = pd.get_dummies(categories)

In [ ]:
print('Shape of data tensor:', data.shape)
print('Shape of labels tensor:', labels.shape)

In [ ]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
#labels = labels.iloc[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]
print('Number of positive and negative reviews in traing and validation set')
print(y_train.columns.tolist())
print(y_train.sum(axis=0).tolist())
print(y_val.sum(axis=0).tolist())

In [ ]:
a=y_train.columns.tolist()
a[18]

In [ ]:
word_index['hi']

In [ ]:
q=x_train[1]
q[1]

### Model

In [ ]:
REG_PARAM = 1e-13
l2_reg = regularizers.l2(REG_PARAM)

In [ ]:
import os

In [ ]:
GLOVE_DIR = "E:/glove/glove.6B.100d.txt"
embeddings_index = {}
f = open(GLOVE_DIR,encoding="utf8")
for line in f:
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        print(word)
        pass
f.close()
print('Total %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, embed_size))
absent_words = 0
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        absent_words += 1
print('Total absent words are', absent_words, 'which is', "%0.2f" % (absent_words * 100 / len(word_index)), '% of total words')

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,embed_size,weights=[embedding_matrix], input_length=max_senten_len, trainable=False)

In [ ]:
word_input = Input(shape=(max_senten_len,), dtype='float32')
word_sequences = embedding_layer(word_input)
word_lstm = Bidirectional(LSTM(150, return_sequences=True, kernel_regularizer=l2_reg))(word_sequences)
word_dense = TimeDistributed(Dense(200, kernel_regularizer=l2_reg))(word_lstm)
word_att = AttentionWithContext()(word_dense)
wordEncoder = Model(word_input, word_att)

sent_input = Input(shape=(max_senten_num, max_senten_len), dtype='float32')
sent_encoder = TimeDistributed(wordEncoder)(sent_input)
sent_lstm = Bidirectional(LSTM(150, return_sequences=True, kernel_regularizer=l2_reg))(sent_encoder)
sent_dense = TimeDistributed(Dense(200, kernel_regularizer=l2_reg))(sent_lstm)
sent_att = Dropout(0.5)(AttentionWithContext()(sent_dense))
preds = Dense(40, activation='softmax')(sent_att)
model = Model(sent_input, preds)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
embedding_layer

In [ ]:
checkpoint = ModelCheckpoint('best_model.h5', verbose=0, monitor='val_loss',save_best_only=True, mode='auto') 

In [ ]:
#x_train = np.asarray(x_train)
#y_train = np.asarray(y_train)
#x_val = np.asarray(x_val)
#y_val = np.asarray(y_val)

In [ ]:
#history = model.fit(x_train, y_train ,validation_data=(x_val, y_val), epochs=10, batch_size=512, callbacks=[checkpoint])

In [ ]:
#model.save('try2.h5')

## Plotting time

In [ ]:
#print(history.history.keys())

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#model.save('han.h5')

In [ ]:
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras import initializers, regularizers, constraints

def dot_product(x, kernel):
    """
    https://github.com/richliao/textClassifier/issues/13#issuecomment-377323318
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    
    Note: The layer has been tested with Keras 2.0.6
    
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """
    
    def __init__(self, return_coefficients=False,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform')
        
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
        
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)
    
    def build(self, input_shape):
        assert len(input_shape) == 3
        
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        
        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)
        
        super(AttentionWithContext, self).build(input_shape)
    
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None
    
    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
        
        if self.bias:
            uit += self.b
        
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
        
        a = K.exp(ait)
        
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        
        a = K.expand_dims(a)
        weighted_input = x * a
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), a]
        else:
            return K.sum(weighted_input, axis=1)
    
    
    
    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]
    def get_config(self):
        base_config = super(AttentionWithContext, self).get_config()
        #base_config['output_dim'] = self.output_dim
        return base_config
   

In [ ]:
test="tech using climate to reduce green house gases drops by 60% and greener fields"

In [ ]:
predict_out(test)

In [ ]:
import pandas as pd
data={'text':[test]}
df3=pd.DataFrame(data)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import multi_gpu_model

In [ ]:
from tensorflow.python import keras
print(keras.__version__)

In [ ]:
y_train.to_pickle("./dummy.pkl")

In [ ]:
from tensorflow.keras.models import load_model

model3=load_model('han.h5',custom_objects={'AttentionWithContext': AttentionWithContext})

In [ ]:
#a=model3.predict(x_train[:4])

In [ ]:
#a1=a[[3]]

In [ ]:
#a

In [ ]:
#b=np.amax(a)

In [ ]:
#b

In [ ]:
#result = np.where(a==b)

In [ ]:
#print(result)

In [ ]:
#p=x_train[:1]

In [ ]:
#p[0]

In [ ]:
#y_train.columns

In [ ]:
#df3

In [ ]:
#df3=pd.read_csv('news_list.csv')

In [ ]:
#text1=df3['text']

In [ ]:
#df3['text']

In [ ]:
sent_lens = []
sent_nums = []
paras1=[]
texts=[]
for idx in range(df3.text.shape[0]):
    text = clean_str(df3.text[idx])
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    sent_nums.append(len(sentences))
    for sent in sentences:
        sent_lens.append(len(text_to_word_sequence(sent)))
    paras1.append(sentences)

In [ ]:
sentences

In [ ]:
sent_lens

In [ ]:
data = np.zeros((len(texts), max_senten_num, max_senten_len), dtype='int32')
for i, sentences in enumerate(paras1):
    for j, sent in enumerate(sentences):
        if j< max_senten_num:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                try:
                    if k<max_senten_len and tokenizer.word_index[word]<max_features:
                        data[i,j,k] = tokenizer.word_index[word]
                        k=k+1
                except:
                    print(word)
                    pass

In [ ]:
data

In [ ]:
indices = np.arange(data.shape[0])
#np.random.shuffle(indices)
data = data[indices]
#labels = labels.iloc[indices]
#nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:2]
#y_train = labels[:-nb_validation_samples]
#x_val = data[-nb_validation_samples:]
#y_val = labels[-nb_validation_samples:]
#print('Number of positive and negative reviews in traing and validation set')
#print(y_train.columns.tolist())
#print(y_train.sum(axis=0).tolist())
#print(y_val.sum(axis=0).tolist())

In [ ]:
x_train

In [ ]:
a=model3.predict(x_train[:2])
a

In [ ]:
b=np.amax(a)

In [ ]:
b

In [ ]:
result = np.where(a==b)


In [ ]:
pos=result[1].item(0)

In [ ]:
pos

In [ ]:
a=y_train.columns.tolist()
a[pos]

In [ ]:
y_train

In [ ]:
from tensorflow.keras.models import load_model

model3=load_model('han.h5',custom_objects={'AttentionWithContext': AttentionWithContext})

In [ ]:
max_features=200000
max_senten_len=40
max_senten_num=6
embed_size=100
VALIDATION_SPLIT = 0.2
import re
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

def predict_out(test):


    data={'text':[test]}
    df3=pd.DataFrame(data)
    sent_lens = []
    sent_nums = []
    paras1=[]
    texts=[]
    for idx in range(df3.text.shape[0]):
        text = clean_str(df3.text[idx])
        texts.append(text)
        sentences = tokenize.sent_tokenize(text)
        sent_nums.append(len(sentences))
        for sent in sentences:
            sent_lens.append(len(text_to_word_sequence(sent)))
        paras1.append(sentences)
        data = np.zeros((len(texts), max_senten_num, max_senten_len), dtype='int32')
    for i, sentences in enumerate(paras1):
        for j, sent in enumerate(sentences):
            if j< max_senten_num:
                wordTokens = text_to_word_sequence(sent)
                k=0
                for _, word in enumerate(wordTokens):
                    try:
                        if k<max_senten_len and tokenizer.word_index[word]<max_features:
                            data[i,j,k] = tokenizer.word_index[word]
                            k=k+1
                    except:
                        #print(word)
                        pass
    indices = np.arange(data.shape[0])
#np.random.shuffle(indices)
    data = data[indices]
#labels = labels.iloc[indices]
#nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

    x_train = data[:2]
    a=model3.predict(x_train[:2])
    #print(a)
    b=np.amax(a)
    #print(b)
    result = np.where(a==b)
    pos=result[1].item(0)
    #print(pos)
    
    a=y_train.columns.tolist()
    
    return a[pos]

In [ ]:
import pickle

# saving
#with open('tokenizer.pickle', 'wb') as handle:
    #pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
y_train = pd.read_pickle("./dummy.pkl")
y_train
# loading
with open('tokenizer.pickle', 'rb') as handle:
      tokenizer = pickle.load(handle)
        

In [ ]:
from flask import Flask, redirect, url_for, render_template, request, session,jsonify

In [ ]:
from werkzeug.wrappers import Request, Response

In [ ]:
app = Flask(__name__, template_folder='./')
app.static_folder = 'static'

@app.route('/prediction', methods=['POST', 'GET'])
def prediction():
    if request.method == "POST":
        message = request.form['message']
        print(message)
        response =  predict_out(message)
        print(response)
        msg="The news category is"
        
        
        return jsonify(msg,response)
    
    return jsonify("Input text")

@app.route('/')
def main():
    return render_template('index1.html')


if __name__ == '__main__':
    #app.run(debug=True)
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)
    
@app.route('/shutdown', methods=['POST'])
def shutdown():
    shutdown_server()
    return 'Server shutting down...'

    

In [ ]:
word_index=tokenizer.word_index

In [ ]:
word_index['bella']

In [ ]:
df2=pd.read_csv('val.csv').reset_index()

In [ ]:
df3=df2[:20]

In [ ]:
predicted=[]

In [ ]:
arr2=[]

In [ ]:
for i in df3['text']:
    arr2.append(predict_out(i))
    

In [ ]:
df3['predicted']=arr2

In [ ]:
txt2="Is The World Ending? How To Explain Mayan Doomsday To Kids With doomsday rumors making the rounds online and likely at the water cooler, those who aren't in the know — that a Mayan"

In [ ]:
df3

In [ ]:
model3.optimizer

In [ ]:
df3=df2[:1000]

In [ ]:
y_true=df3['category']

In [ ]:
y_predicted=df3['predicted']

In [ ]:
y_predicted

In [ ]:
cates = df2.groupby('category')

from keras import backend as K
K.eval(model3.optimizer.lr)

In [ ]:
print(cates)

In [ ]:
cf_matrix = confusion_matrix(y_true, y_predicted)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
from sklearn import metrics

In [ ]:
cf_matrix

In [ ]:
cols = ['ARTS', 'ARTS & CULTURE', 'BLACK VOICES', 'BUSINESS', 'COLLEGE', 'COMEDY', 'CRIME', 'CULTURE & ARTS', 'DIVORCE', 'EDUCATION', 'ENTERTAINMENT', 'ENVIRONMENT', 'FIFTY', 'FOOD & DRINK', 'GOOD NEWS', 'GREEN', 'HEALTHY LIVING', 'HOME & LIVING', 'IMPACT', 'LATINO VOICES', 'MEDIA', 'MONEY', 'PARENTING', 'PARENTS', 'POLITICS', 'QUEER VOICES', 'RELIGION', 'SCIENCE', 'SPORTS', 'STYLE', 'STYLE & BEAUTY', 'TASTE', 'TECH', 'TRAVEL', 'WEDDINGS', 'WEIRD NEWS', 'WELLNESS', 'WOMEN', 'WORLD NEWS', 'WORLDPOST']

In [ ]:
plt.figure(figsize=(10,10))
heatmap = sns.heatmap(cf_matrix, xticklabels=cols,
                      yticklabels=cols,
                      annot=True, fmt='d', color='blue')
plt.xlabel('Predicted class')
plt.ylabel('True class')
plt.title('Confusion matrix of model')